# Exploring USA Gas Pipeline Accident Data

The United States maintains over 3.2 million kilometers of natural gas distribution mains and pipelines, over 500,000 kilometers of gas transmission and gathering pipelines, over 180,000 kilometers hazardous liquid pipeline, and 114 active liquid natural gas plants that are connected to natural gas transmission and distribution systems [<sup>1</sup>](https://www.ncsl.org/research/energy/state-gas-pipelines.aspx#:~:text=The%20United%20States%20maintains%20about,gas%20transmission%20and%20distribution%20systems.). 

The pipeline system includes:

 * __Transmission lines__ to transport large quantities of natural gas or hazardous liquids over long distances from gathering lines or storage facilities to distribution centers, storage facilities, power plants, industrial customers and municipalities. Most transmission pipelines are located underground.
 
 * __Distribution lines__ to move gas to industrial customers. Smaller distribution lines connect businesses and homes. Distribution lines usually are installed in underground utility easements along streets. Gas pipeline *commodities* include natural gas, hydrogen gas, propane gas and synthetic gas. Almost all gas gathering lines are for natural gas. Distribution and transmission are mostly for natural gas, but include some propane and hydrogen.
 
In this project we explore the incient data provided by the [Pipeline and Hazardous Materials Safety Administration](https://www.phmsa.dot.gov/data-and-statistics/pipeline/distribution-transmission-gathering-lng-and-liquid-accident-and-incident-data) to explore the locations, causes and asspcoated casualties of gas pipeline incidents in the United States over the last 10 years.We utilise [plotly](https://plotly.com/) to make interactive visualisations of the data. 

In [ ]:
#importing the 
import pandas as pd

#import wget

#plotty things
import plotly
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.offline import init_notebook_mode
import plotly.express as px

import pyproj

#mappy things
#import geopandas as gpd
#import shapely.geometry # to make the interactive plotly pipe shapes work 
#from shapely.geometry import Point  #To display lat-long locations on the maps


pd.options.display.max_columns = 999


# Pipeline Geographical Locations 
The figures below show the geographical locations of the gas pipelines. The data associated with these can viwed by hovering over the individual shape. 
The pipeline representations are in three parts, the __Transmission lines__, the __Distribution lines__, and finally the __Gulf of Mexico lines__. 

In [ ]:
# Loading the converted shape files. The shape files have to be converted from their geometric multiline representations to lat/lon 
# representations of the two ends of each of the segments. The algorithem to do this is in the previous versions of the code in the github repository. 
# You can simply uncomment each cell and have the .pkl variable constructed again. It just takes a little bit fo time, hence the 
# loading of the variable to save time.  
geo_df_trans = pd.read_pickle('geo_df_trans.pkl') #Gas transmission lines file
geo_df = pd.read_pickle('geo_df.pkl') # Gas distribution lines file, its the largest
geo_df_mx = pd.read_pickle('geo_df_mx.pkl') #Gas pipelines in the Gulf of Mexico

In [ ]:
#Plotting the dataframe we made with lats, lons and names etc form the HGL, Inter/Intra and Gulf of Mexico shape files, 
# we manipulate the hoverdata to display what we like.


# GULF OF MEXICO
fig = px.line_geo(geo_df_mx,lat = geo_df_mx.lats, lon = geo_df_mx.lons, hover_name = geo_df_mx.Operator_Name, locationmode="USA-states", scope="usa",
                 hover_data={'lats':False, #remove latitude form hover data
                             'lons':False, #remove longitude form hover data
                             'Category':False, #remove category from hover data
                              },
                              color = 'Category' )

fig.update_traces(line_color='#8723e3', line_width=0.25) #updating the default lines used for HGL piepes


# #--------------------------------------
#Inter/Intra State Pipelines
fig2 = px.line_geo(geo_df,lat = geo_df.lats, lon = geo_df.lons, hover_name = geo_df.Operator_Name, locationmode="USA-states", scope="usa",
                 hover_data={'lats':False, #remove latitude form hover data
                             'lons':False, #remove longitude form hover data
                             'Category':False, #remove category from hover data
                            },
                               color = 'Category' )
fig2.update_traces(line_color='#293d6e', line_width=0.25) #updating the default lines used for HGL piepes
# #--------------------------------------

#HGL Transmission Lines
#plotting the shape file onto the USA map using plotly
fig3 = px.line_geo(geo_df_trans,lat = geo_df_trans.lats, lon = geo_df_trans.lons, hover_name = geo_df_trans.Pipe_Name, locationmode="USA-states", scope="usa",
                 hover_data={'lats':False, #remove latitude form hover data
                             'lons':False, #remove longitude form hover data
                             'Operator_Name': True, 
                             'Category':False, #remove category from hover data
                            },
                               color = 'Category')
fig3.update_traces(line_color='#000000', line_width=.75) #updating the default lines used for HGL piepes
#
#Adding all the traces together in one map here
fig.add_trace(fig2.data[0]) 
fig.add_trace(fig3.data[0]) 
# #--------------------------------------


#fig.update_layout()
fig.update_layout(title = 'USA Gas Pipelines',
                  hovermode="closest",
                  margin={"r":0,"t":0,"l":0,"b":0},
                  showlegend = True)


fig.update_geos(
    resolution=50,
    showland=True, landcolor="#b1f699",
    showlakes=True, lakecolor="LightBlue")

fig.show()



# Loading and Filtering the Gas Transmission Lines Incident Data

Now that we have some knowledge about the location of the various types of gas pipelines, we take a closer look at the incident data associated with these.
We start with the gas transmission pipelines. 

In [ ]:
#changed encoding by saving as csv file again, tab delimited
HGL_trans_accidents = pd.read_csv('data/incident_gas_transmission_gathering_jan2010_present.csv',low_memory=False) 
HGL_trans_accidents.shape

The data file we have has so many columns that are not useful, so I have made a .txt file of the columns required for our analysis. It is saved in the same directory.

In [ ]:
# Reading in the required columns form the text file. I have kept it as a text file so it is easy to modify for future uses. 
req_clmns = []

#importing the list of required columns form the text file
with open('data/Gas_Trans_required_columns.txt') as f: 
    lines = f.readlines()
f.close()

for l in lines:
    req_clmns.append(l.strip()) #getting rid of the "\n" escape character

HGL_trans_acc = HGL_trans_accidents[req_clmns]
# I have discovered some outliers that fall outside the continental USA, so I will explude those: 
HGL_trans_acc = HGL_trans_acc[(-140<HGL_trans_acc['LOCATION_LONGITUDE']) & (HGL_trans_acc['LOCATION_LONGITUDE']<-50)]
HGL_trans_acc = HGL_trans_acc[HGL_trans_acc['LOCATION_LATITUDE']<50]

HGL_trans_acc.shape # we moved from  625 columns to 61
HGL_trans_acc.sample(3)


In [ ]:
HGL_trans_acc.REPORT_NUMBER.unique

# Exploring the Incident Data Using Plotly
To do this we make a plotly bar plot with incident count, year and state

In [ ]:
#Grouping the dataframe by year and then state counts, and sorting it.

grouped_Year_State = HGL_trans_acc.groupby(['IYEAR','OPERATOR_STATE_ABBREVIATION']).count().sort_values(by = 'REPORT_NUMBER',  ascending=False).reset_index()
grouped_Year_State.head()

# The column report number (1 on 1 relationship) can be our incident count so:

grouped_Year_State.rename(columns = {'IYEAR':'Incident Year','OPERATOR_STATE_ABBREVIATION':'Incident State', 'REPORT_NUMBER':'No of Incidents'}, inplace=True)
grouped_Year_State.sample(3)
grouped_Year_State['No of Incidents'].sum()

In [ ]:
# Plotting the data using plotly:
fig = px.bar(data_frame=grouped_Year_State, y = 'Incident Year', x = 'No of Incidents',
            color = 'Incident State', orientation='h', 
            color_discrete_sequence= px.colors.sequential.deep_r,
            title='Number of Gas Transmission Line Incidents in Past 10 Years by State') #remove latitude form pipe)

fig.update_layout(showlegend=True)
fig.show()


Good reference for the colour pallets. Only the sequential one works here. 
- dir(px.colors.qualitative)
- dir(px.colors.diverging)
- dir(px.colors.sequential)

## Exploring the Injuries, Fatalities and Cuases of Gas Transmission Line Incidents
### Fatalities and Injuries

In [ ]:
# Calculating the number of fatalities
fatality = HGL_trans_acc['FATAL'].value_counts().reset_index()
fatality.rename(columns={'index':'Fatalities'}, inplace = True)
fatality['deaths'] = fatality['Fatalities']*fatality['FATAL']
# print('The fatality count is:')
# print(fatality['deaths'].sum())
Deaths = fatality['deaths'].sum()

# Calculating the number of injuries
injury = HGL_trans_acc['INJURE'].value_counts().reset_index()
injury.rename(columns={'index':'Injuries'}, inplace = True)
injury['injuries'] = injury['Injuries']*injury['INJURE']
# print('The injury count is:')
# print(injury['injuries'].sum())

Injuries = injury['injuries'].sum()

data = {'Death':fatality['deaths'].sum(),'Injury':injury['injuries'].sum()}
Casualties=pd.DataFrame(data,index=['Number of Casualties'])

Casualties

We can see that over the last 10 years, a total of 31 deaths and 117 injuries have occured. Given that the transmission lines are generally not in populated areas and are not very widespeard, this low count is expected. We will compare this with the far more extensive and widespread distribution netwrok.

### Causes of the Gas Transmission Line Incidents

In [ ]:
Acc_Cause = HGL_trans_acc.groupby(['CAUSE','CAUSE_DETAILS']).count().reset_index()
Acc_Cause.rename(columns = {'REPORT_NUMBER':'Number of Incidents','CAUSE':'Accident Cause', 'CAUSE_DETAILS':'Details'}, inplace=True) #each incident has a report number so its count is incident count

Acc_Cause['Accident Cause'] = Acc_Cause['Accident Cause'].str.lower().str.title()
Acc_Cause['Details'] = Acc_Cause['Details'].str.lower().str.capitalize()

Acc_Cause.head(3)


In [ ]:

# #trying with plotly express
fig = px.bar(data_frame=Acc_Cause, y = 'Accident Cause', x = 'Number of Incidents', color = 'Details',
            color_discrete_sequence= px.colors.sequential.Bluyl_r, orientation='h', 
            title='Causes of Gas Transmission Line Incidents in Past 10 Years',hover_data={'Accident Cause':False}) #remove latitude form pipe)
fig.update_layout(showlegend=False)
fig.show()


We can see that the main cause for the incidents is equiment failure followed by corrosion failure. To get a better understanding of the share of each incident cause we can use a pie chart:

In [ ]:
fig = px.pie(Acc_Cause, values='Number of Incidents', names='Accident Cause',
             color_discrete_sequence= px.colors.sequential.Bluyl,
             title = 'Gas Distribution Indicent Cause Breakown')
fig.show()

# Incidents in the Gas Distribution Network 

In [ ]:
# Working on Gas Distribution Accidents now:
#changed encoding by saving as csv file again, tab delimited
Gas_Dist_Accidents = pd.read_csv('data/incident_gas_distribution_jan2010_present.csv',low_memory=False) 
Gas_Dist_Accidents.sample(2)
Gas_Dist_Accidents.shape


In [ ]:
req_clmns =  []
with open('data/Gas_Dist_required_columns.txt') as f: #importing the list of required columns form the text file
    lines = f.readlines()
f.close()
for l in lines:
    req_clmns.append(l.strip()) #getting rid of the "\n" escape character
Gas_Dist_Acc = Gas_Dist_Accidents[req_clmns]

Gas_Dist_Acc.shape
Gas_Dist_Acc.sample(3)

There is an outlier in the longitude column (large negative number), so we find it and drop that whole row so we can do the plotting 

In [ ]:
drop_row_index = Gas_Dist_Acc['LOCATION_LONGITUDE'].idxmin() #getting the row index for the minimum oulier 
Gas_Dist_Acc.drop(Gas_Dist_Acc.index[drop_row_index])
Gas_Dist_Acc.shape #So we dropped one row

Gas Distribution netwrok Incidents' Geographical Locations

In [ ]:
# I have discovered some outliers that fall outside the continental USA, so I will explude those: 
Gas_Dist_Acc = Gas_Dist_Acc[(-140<Gas_Dist_Acc['LOCATION_LONGITUDE']) & (Gas_Dist_Acc['LOCATION_LONGITUDE']<-50)]
Gas_Dist_Acc = Gas_Dist_Acc[Gas_Dist_Acc['LOCATION_LATITUDE']<50]

#Formatting  the hovertext
Gas_Dist_Acc['hover_text'] = Gas_Dist_Acc["LOCATION_CITY_NAME"].str.title()+", "+ Gas_Dist_Acc["LOCATION_STATE_ABBREVIATION"]+ ", "+"Cause: "+Gas_Dist_Acc['CAUSE'].str.lower().str.capitalize()+", " + Gas_Dist_Acc["IYEAR"].astype(str)

fig = go.Figure(data = go.Scattergeo(
                    lat=Gas_Dist_Acc.LOCATION_LATITUDE, lon=Gas_Dist_Acc.LOCATION_LONGITUDE,
                    text=Gas_Dist_Acc['hover_text'], 
                    mode = 'markers',
                    marker = dict(
                    size = 4,
                    opacity = 0.8,
                    reversescale = True,
                    autocolorscale = False,
                    symbol = 'circle', 
                    line = dict(
                        width=1,
                        color='rgba(219, 15, 15)'
                    ),
                    colorscale = 'Hot_r',
                    cmin = 2010,
                    color = Gas_Dist_Acc.IYEAR,
                    cmax = Gas_Dist_Acc.IYEAR.max(),
                    colorbar_title="Incident Year")))

fig.update_layout(
        title = 'Gas distribution line incidents in the last 10 years',
        geo_scope='usa',
        hovermode="closest",
        margin={"r":0,"t":0,"l":10,"b":0},
        showlegend = True
    )


fig.update_geos(
    resolution=50,
    showland=True, landcolor="#b1f699",
    showlakes=True, lakecolor="LightBlue")

fig.show()


In [ ]:
Gas_Dist_Acc.shape #So we dropped one row

In [ ]:
#Using Scatter Mapbox for a different type of location plot

fig = px.scatter_mapbox(Gas_Dist_Acc, lat="LOCATION_LATITUDE", lon="LOCATION_LONGITUDE", hover_name="LOCATION_CITY_NAME",
                    hover_data=["CAUSE_DETAILS", "IYEAR"],
                    color_discrete_sequence=["red"], zoom=3, height=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
grouped_Year_State = Gas_Dist_Acc.groupby(['IYEAR','LOCATION_STATE_ABBREVIATION']).count().sort_values(by = 'NAME',  ascending=False).reset_index()

#creating a data frame as the output of the groupby. The product of the groupby is a series, 
# so you do a to_frame to get the df and then reset the index to get a proper index
# The column NAME (1 on 1 relationship) can be our incident count so:

grouped_Year_State.rename(columns = {'IYEAR':'Incident Year','LOCATION_STATE_ABBREVIATION':'Incident State', 'NAME':'No of Incidents'}, inplace=True)
#Sorting based on year
grouped_Year_State=grouped_Year_State.sort_values("Incident Year") # Make sure you sort the time horizon column in ascending order

grouped_Year_State.sample(3)



In [ ]:
#plotting using plotly:

# #trying with plotly express
fig = px.bar(data_frame=grouped_Year_State, y ='No of Incidents' , x = 'Incident State',
            color = 'Incident Year', orientation='v', 
            color_continuous_scale=px.colors.sequential.OrRd_r,
            title='Number of Gas Distribution Incidents in Past 10 Years by State') #remove latitude form pipe)

fig.update_layout(showlegend=True)
fig.show()


The above is not very telling and difficult to interpret even with the hover text. So we do a Chorlopleth Map for every year. 

### Animated Chorlopleth Map to understand the gas distribution incident data better.
https://towardsdatascience.com/simplest-way-of-creating-a-choropleth-map-by-u-s-states-in-python-f359ada7735e

In [ ]:
import plotly.express as px
fig = px.choropleth(grouped_Year_State,
                    locations='Incident State', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='No of Incidents',
                    color_continuous_scale="agsunset_r", 
                    animation_frame='Incident Year' #make sure 'Incident Year' is string type and sorted in ascending order
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

# Explorting the Injuries, Fatalities and Cuases of the Gas Transmission Netwrok Incidents
## Fatalities and Injuries

In [ ]:
# Calculating the number of fatalities
fatality = Gas_Dist_Acc['FATAL'].value_counts().reset_index()
fatality.rename(columns={'index':'Fatalities'}, inplace = True)
fatality['deaths'] = fatality['Fatalities']*fatality['FATAL']
print('The fatality count is:')
print(fatality['deaths'].sum())

# Calculating the number of injuries
injury = Gas_Dist_Acc['INJURE'].value_counts().reset_index()
injury.rename(columns={'index':'Injuries'}, inplace = True)
injury['injuries'] = injury['Injuries']*injury['INJURE']
print('The injury count is:')
print(injury['injuries'].sum())

The injury/fatality rate is larger than the Gas Transmission lines injury/fatality, so this warrants a closer look. We combine the fatalities and the injuries into a new column "Casualties". The reason for the larger number of casualties, is that the gas distribution netwrok has a considerable presence in locations with higher population densities. As a result, the casualties are higher when there is an incident. 

In [ ]:

Casualties = Gas_Dist_Acc[(Gas_Dist_Acc['FATALITY_IND']=='YES')|(Gas_Dist_Acc['INJURY_IND']=='YES')]
Casualties_by_State = Casualties.groupby(['LOCATION_STATE_ABBREVIATION', 'CAUSE', 'CAUSE_DETAILS']).sum()
Casualties_by_State= Casualties_by_State.reset_index()
Casualties_by_State.rename(columns = {'LOCATION_STATE_ABBREVIATION':'State',
                                    'FATAL':'Number_Fatalities',
                                    'CAUSE': 'Cause',
                                    'CAUSE_DETAILS': 'Cause Details',
                                    'INJURE':'Number_Injuried'}, inplace=True)


Casualties_by_State['Cause'] = Casualties_by_State['Cause'].str.lower().str.capitalize()
Casualties_by_State['Cause Details'] = Casualties_by_State['Cause Details'].str.lower().str.capitalize()
Casualties_by_State['Casualties'] = Casualties_by_State['Number_Fatalities']+Casualties_by_State['Number_Injuried']

Casualties_by_State.sample(3)

In [ ]:

fig = px.bar(data_frame=Casualties_by_State, y ='Casualties' , x ='State' , color ='Cause' , #hover_data='CAUSE_DETAILS',
            color_discrete_sequence= px.colors.sequential.Jet, orientation='v',
            hover_data={'Cause Details'},
            title='Casualties of Gas Distribution Line Incidents in Past 10 Years')
fig.update_layout(showlegend=True)
fig.show()



Using a map to show the worst States in terms of the total casualties in the last 10 years. 

In [ ]:
Casualties_by_State =Casualties_by_State.groupby(['State']).sum().reset_index() #grouping based on States only for the heatmap

In [ ]:
#Plotting the heatmap
fig = px.choropleth(Casualties_by_State,
                    locations='State', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='Casualties',
                    color_continuous_scale="speed", 
                    )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

# Conclusions

In this project we explored the incient data provided by the [Pipeline and Hazardous Materials Safety Administration (PHMSA)](https://www.phmsa.dot.gov/data-and-statistics/pipeline/distribution-transmission-gathering-lng-and-liquid-accident-and-incident-data) and found some insights related to the locations, causes and associated casualties of gas pipeline incidents in the United States over the last 10 years. We utilised [plotly](https://plotly.com/) to make interactive visualisations of the data to help us understand the available data better. 

Causes, casualties, and incident locations are inherently different between gas transmission and gas distribution pipelines. 

The former has less casualities as its not distributed among the areas wiht large population, natually the highest number of incidents occure in Texas, Oklahoma and Gulf of mexico where the transmission lines are more pervalent. The number of incidets per year is stead around 100 incidents. The major causes for gas transmission lines are equipment failure, mainly vause of malfunctioning control and releif equipment, followed by corrosion issues. 

Gas distribution pipeline incidents on the oter hand occure in the locations with much larger population dentisy. As a result their casualty rates are higher. The pipeline netwrok itself is much more widespread as well, contributing to the higher incident and casualty rate. California has the highest number of gas distribution incidents in the last 10 years followed by New York, however, in temrs of casualties, the State of New York has the highest number of casualties. The main cotrnobuter to the incidents resulting in casulties is natural foce damage due to earth movement.

